# Getting Started with a Movie Recommendation System

Reference: https://www.kaggle.com/code/ibtesama/getting-started-with-a-movie-recommendation-system/notebook?select=tmdb_5000_credits.csv

## Prerequisites

- Install Dependencies
- Install Dataset

In [ ]:
%pip install kagglehub
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install scikit-learn

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tmdb/tmdb-movie-metadata")

print("Path to dataset files:", path)

## Approaches

### Demographic Filtering

- 영화 인기도나 장로에 기반하여 모든 사용자에게 일반화된 추천 제공
- 인구통계학적으로 비슷한 사용자에게는 같은 영화 추천
- 더 인기있는 영화가 평균적인 대중들이 좋아할 확률이 더 높을 것이라는 가정에 기반

### Content Based Filtering

- 특정 아이템에 기반하여 비슷한 아이템을 추천
- 장르, 감독, 설명, 배우 등의 메타데이터 사용
- 특정 영화를 좋아하는 사람이러면 비슷한 영화도 좋아할 것이라는 가정에 기반

### Collaborative Filtering

- 비슷한 관심사를 가진 사람들을 매칭하여 추천 제공
- Content Based Filter과 달리 메타데이터를 사용하지 않음

## Load Data

- 앞서 다운로드 받은 데이터를 Pandas DataFrame으로 로드

In [ ]:
import pandas as pd

df1 = pd.read_csv(f"{path}/tmdb_5000_credits.csv")
df2 = pd.read_csv(f"{path}/tmdb_5000_movies.csv")

In [ ]:
# 데이터셋 살펴보기: tmdb_5000_credits.csv
df1

In [ ]:
# 데이터셋 살펴보기: tmdb_5000_movies.csv
df2

## Dataset Join

- `df1`의 일부 컬럼만 뽑아 `df2`와 조인

In [ ]:
df1.columns = ["id", "tittle", "cast", "crew"]
df2 = df2.merge(df1, on="id")

# 5개의 Row 뽑아보기
df2.head(5)

## Demographic Filtering

### TODO

- 영화의 점수를 매길 수 있는 Metric 필요
- 모든 영화의 점수 계산
- 점수에 따라 정렬하고, 사용자에게 가장 높은 점수의 영화 추천


### Metric

- 8.9점이지만 3명만 평가한 영화와 7.8점이지만 40만명이 평가한 영화 중 어느 것이 더 좋은 영화일까?
- 이는 알 수 없으므로, IMDB의 가중 평점(WR) 사용

$$
WR = \left( \frac{v}{v+m} \cdot R \right) + \left( \frac{m}{v+m} \cdot C \right)
$$

- $v$ : 영화에 대한 평가 수
- $m$ : 최소 평가 수
- $R$ : 영화 평균 평점
- $C$ : 전체 영화 평균 평점

이미 $v$와 $R$은 주어져 있으므로, $C$와 $m$을 정의해야 함



In [ ]:
C = df2['vote_average'].mean()
C

- 90th Percentile을 사용하여 $m$ 정의
- 영화가 차트에 추천되기 위해서는, 최소 90%의 영화보다 더 많은 평가를 받아야 함

In [ ]:
m = df2['vote_count'].quantile(0.9)
m

- 대상이 되는 영화 필터링
- 481개의 영화만 추천

In [ ]:
q_movies = df2.copy().loc[df2["vote_count"] >= m]
q_movies.shape

- Weighted Rating 계산

In [ ]:
def weighted_rating(x, m=m, C=C):
    v = x["vote_count"]
    R = x["vote_average"]

    return (v / (v + m) * R) + (m / (m + v) * C)

In [ ]:
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

- Score를 기반으로 정렬

In [ ]:
q_movies = q_movies.sort_values('score', ascending=False)

q_movies[['title', 'vote_count', 'vote_average', 'score']].head(10)

In [ ]:
pop = df2.sort_values("popularity", ascending=False)

import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

plt.barh(
    pop["title"].head(6), pop["popularity"].head(6), align="center", color="skyblue"
)
plt.gca().invert_yaxis()
plt.xlabel("Popularity")
plt.title("Popular Movies")

## Content Based Filtering

- 영화의 개요, 감독, 배우, 장르 등을 사용하여 영화 간 유사성 계산
- 유사성 점수를 기반으로 추천
- `overvie` 컬럼을 사용하여 영화 간 유사성 계산
- Term Frequency-Inverse Document Frequency(TF-IDF) 사용

In [ ]:
# Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

# Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words="english")

# Replace NaN with an empty string
df2["overview"] = df2["overview"].fillna("")

# Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df2["overview"])

# Output the shape of tfidf_matrix
tfidf_matrix.shape

- Cosine Similarity 사용하여 유사성 계산

$$
cosine(x, y) = \frac{x \cdot y}{||x|| \cdot ||y||}
$$

- TF-IDF vectorizer를 사용했기 때문에, Dot Product가 Cosine Similarity와 동일

In [ ]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
indices = pd.Series(df2.index, index=df2['title']).drop_duplicates()

- Recommendation 함수 구현
- 영화 제목으로부터 인덱스 추출
- 모든 영화와의 Cosine Similarity 계산하고, Position과 Score 튜플로 변환
- Score에 따라 정렬하고, 상위 10개 추천

In [ ]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df2['title'].iloc[movie_indices]

In [ ]:
get_recommendations('The Dark Knight Rises')